# Refinement
This notebook will show how to perform a simple form of structure refinement

## Generate artificial data

We will here generate some scattering data, to refine against

In [7]:
from iotbx import pdb

# Load structure, use a pdb of tyrosine
xrs = pdb.input(
    source_info="", 
    lines="""
CRYST1   16.170   14.591   15.187  90.00  90.00  90.00 P 1
SCALE1      0.061843  0.000000  0.000000        0.00000
SCALE2      0.000000  0.068535  0.000000        0.00000
SCALE3      0.000000  0.000000  0.065846        0.00000
ATOM      1  N   TYR A   4       8.357   9.217   8.801  1.00 10.55           N
ATOM      2  CA  TYR A   4       9.150   8.055   9.050  1.00 10.24           C
ATOM      3  C   TYR A   4      10.419   8.399   9.804  1.00  9.86           C
ATOM      4  O   TYR A   4      10.726   9.591  10.050  1.00 11.39           O
ATOM      5  CB  TYR A   4       9.496   7.352   7.737  1.00 30.00           C
ATOM      6  CG  TYR A   4       8.296   6.791   7.006  1.00 30.00           C
ATOM      7  CD1 TYR A   4       7.820   5.517   7.293  1.00 30.00           C
ATOM      8  CD2 TYR A   4       7.642   7.534   6.034  1.00 30.00           C
ATOM      9  CE1 TYR A   4       6.724   5.000   6.628  1.00 30.00           C
ATOM     10  CE2 TYR A   4       6.545   7.024   5.364  1.00 30.00           C
ATOM     11  CZ  TYR A   4       6.091   5.758   5.665  1.00 30.00           C
ATOM     12  OH  TYR A   4       5.000   5.244   5.000  1.00 30.00           O
ATOM     13  OXT TYR A   4      11.170   7.502  10.187  1.00  9.86           O
ATOM     14  H1  TYR A   4       8.254   9.323   7.923  1.00 10.55           H
ATOM     15  H2  TYR A   4       7.560   9.120   9.184  1.00 10.55           H
ATOM     16  H3  TYR A   4       8.764   9.932   9.140  1.00 10.55           H
ATOM     17  HA  TYR A   4       8.637   7.441   9.599  1.00 10.24           H
ATOM     18  HB2 TYR A   4       9.929   7.989   7.148  1.00 30.00           H
ATOM     19  HB3 TYR A   4      10.097   6.615   7.928  1.00 30.00           H
ATOM     20  HD1 TYR A   4       8.245   5.005   7.942  1.00 30.00           H
ATOM     21  HD2 TYR A   4       7.946   8.389   5.830  1.00 30.00           H
ATOM     22  HE1 TYR A   4       6.415   4.146   6.829  1.00 30.00           H
ATOM     23  HE2 TYR A   4       6.116   7.532   4.714  1.00 30.00           H
ATOM     24  HH  TYR A   4       4.712   5.804   4.444  1.00 30.00           H
END
""",
).xray_structure_simple()
xrs.scattering_type_registry(table="electron")

# Generate Fobs
f_obs = xrs.structure_factors(d_min=0.8).f_calc().as_amplitude_array()
r_free_flags = f_obs.generate_r_free_flags_basic()

# Shake the structure a little, to have something to refine against
xrs.shake_sites_in_place(rms_difference=0.1)
# xrs.shake_adp()

True

## Refinement loop
Here we show a simple gradient descent refinement with gradients from pydiscamb

In [ ]:
import pydiscamb

# Set up wrapper
wrapper = pydiscamb.DiscambWrapper(xrs)
wrapper.set_indices(f_obs.indices())

# Set up target functor
from cctbx.array_family import flex
from cctbx.xray import target_functors
lsq = target_functors.least_squares(
    compute_scale_using_all_data=True, 
    f_obs=f_obs, 
    weights=flex.double(f_obs.size(), 1.0), 
    r_free_flags=r_free_flags, 
    scale_factor=1,
)

for cycle in range(10):
    # Compute gradients
    f_calc = f_obs.array(data = wrapper.f_calc())
    print(f_obs.r1_factor(f_calc))
    d_target_d_f_calc = lsq(f_calc, False).target_per_reflection()
    grads = wrapper.d_target_d_params(list(d_target_d_f_calc))

    # Update structure
    print(xrs.scatterers()[0].site)
    for grad, scatterer in zip(grads, xrs.scatterers()):
        scatterer.site = (
            scatterer.site[0] - 0.01 * grad.site_derivatives[0],
            scatterer.site[1] - 0.01 * grad.site_derivatives[1],
            scatterer.site[2] - 0.01 * grad.site_derivatives[2],
        )
    print(xrs.scatterers()[0].site)
    wrapper.update_structure(xrs)


0.1005064515328304
(0.5165253338155734, 0.63597654839914, 0.578988328425782)
(0.5169212425706852, 0.6351576828089455, 0.5787618835237209)
0.1290907633056445
(0.5169212425706852, 0.6351576828089455, 0.5787618835237209)
(0.5119262877231266, 0.6389662347346619, 0.5790378497707083)
0.28526525044004686
(0.5119262877231266, 0.6389662347346619, 0.5790378497707083)
(0.47904933788541926, 0.6446018143526926, 0.5827464904414766)
0.5482311731073953
(0.47904933788541926, 0.6446018143526926, 0.5827464904414766)
(0.4602919204154934, 0.4241549203716044, 0.6713423675931709)
0.5892799803046243
(0.4602919204154934, 0.4241549203716044, 0.6713423675931709)
(0.14648048221677462, 0.2958990460191544, 0.9396279093005958)
0.6331462920534975
(0.14648048221677462, 0.2958990460191544, 0.9396279093005958)
(-0.3114926304085149, -0.3772191516548208, 1.7691553030987461)
0.6322039488209129
(-0.3114926304085149, -0.3772191516548208, 1.7691553030987461)
(-0.10210159884793163, 1.0691540955221437, 1.9388677352578523)
0.638

In [9]:
f_obs.r

AttributeError: 'array' object has no attribute 'r'

In [ ]:
dpdfc = lsq(f_obs.array(data=flex.complex_double(f_obs.size(), 1.0)), False)

In [ ]:
dir(dpdfc)

In [ ]:
dpdfc.target_per_reflection()